In [36]:
import yfinance as yf
import pandas as pd
import requests, csv, json
from datetime import datetime, timedelta

### Get list of all active tickers from all exchanges as of date

In [37]:
def get_all_active_stocks():
    
    CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=TYTNO3561Z0C1XA2'

    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)
        df = pd.DataFrame(my_list)
        headers = df.iloc[0]
        listings = pd.DataFrame(df.values[1:], columns=headers)
        listings = listings.set_index('symbol')
    return listings
    

listings = get_all_active_stocks()

In [47]:
# saving all listing as of today
listings.to_csv('listings.csv')

# saving counts by exchanges
listings[['exchange','assetType']] \
    .value_counts() \
    .reset_index(name='count') \
    .to_csv('listings_ct.csv', index=False)

# selecting tickers from NASDAQ  
tickers = list(listings[(listings['exchange'] == 'NASDAQ') & (listings['assetType'] == 'Stock')].index.values)

print('selected {0} tickers from NASDAQ'.format(len(tickers)))

selected 4820 tickers from NASDAQ


In [48]:
tickers

['AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AADI',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABIO',
 'ABNB',
 'ABOS',
 'ABSI',
 'ABST',
 'ABUS',
 'ABVC',
 'ACAB',
 'ACABU',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACAX',
 'ACAXR',
 'ACAXU',
 'ACAXW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACCD',
 'ACDC',
 'ACDCW',
 'ACER',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACGN',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACMR',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACOR',
 'ACRS',
 'ACRV',
 'ACRX',
 'ACST',
 'ACT',
 'ACTG',
 'ACVA',
 'ACXP',
 'ADAG',
 'ADAL',
 'ADALU',
 'ADALW',
 'ADAP',
 'ADBE',
 'ADD',
 'ADEA',
 'ADER',
 'ADERU',
 'ADERW',
 'ADES',
 'ADI',
 'ADIL',
 'ADILW',
 'ADMA',
 'ADMP',
 'ADN',
 'ADNWW',
 'ADOC',
 'ADOCR',
 'ADOCW',
 'ADP',
 'ADPT',
 'ADSE',
 'ADSEW',
 'ADSK',
 'ADTH',
 'ADTHW',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'ADXN',
 'AEAE',
 'AEAEU',
 'AEAEW',
 'AEHA',
 'AEHL'

# 1. Rocket stocks

Rocket stocks follow below trend (only works in bullish market) 
1. Hits all time highs everyday
    * At ATHs all buyers are happy
    * short sellers also will be forced to buy when ATHs hit consistently 
    * panicked buying kicks in 
2. Surge in Daily volume and stock gaps occur
    * check stocks that are trading at 4 - 40x daily averge volume
3. Stock trades above 50 day MA
4. 50 day MA > 200 day MA
5. Market cap < 4B 
6. Float is 20% or less of Total shares. 
    * Float is total available shares to trade
7. Short interest > 10% 
8. Check for bullish market - SPY trading above 50 MA and 50MA > 200 MA


### Download data for all tickers

In [4]:
df = yf.download(tickers='NVDA', period='30d', interval='1d')
df

In [5]:
tickers = yf.Tickers(tickers)

In [51]:
stock_info = {}

for stock in tickers:
    try:
        # Create a Ticker object for the stock
        ticker = yf.Ticker(stock)
        stock_info[stock] = ticker.info
    except Exception as e:
        # Handle any other exceptions
        print(f"An error occurred for {stock}: {str(e)}")
    
    continue


An error occurred for AEHA: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AEHA?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for AGFS: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AGFS?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for AKUS: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AKUS?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for ARCK: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ARCK?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for ARCKU: 404

An error occurred for HMCOW: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HMCOW?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for HORIU: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HORIU?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for IMPP-V: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IMPP-V?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for INKA: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/INKA?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for IN

An error occurred for PFXNL: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PFXNL?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for PONOU: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PONOU?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for RAMMU: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RAMMU?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for RNERU: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RNERU?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true
An error occurred for SC

In [52]:
df = pd.DataFrame(stock_info).T

In [54]:
df.to_csv('stock_info.csv')

In [9]:
from pandas_datareader import data

for stock in tickers:
    end_date = datetime.now()
    start_date = end_date - timedelta(days=2*365)
    hist = yf.download(tickers=stock, start=start_date, end=end_date, interval='1d')
    
    ticker = yf.Ticker(stock)
    stock_info = ticker.info
    print(stock_info)
    
    hist['stock_float'] = stock_info.get('floatShares')
    hist['short_interest'] = stock_info.get('shortInterest')
    
    hist['prev_Close'] = hist.loc[:, 'Close'].shift(1)
    hist['prev_Volume'] = hist.loc[:, 'Volume'].shift(1)
#     hist['200SMA'] = hist['prev_Close'].rolling(200).mean()
#     hist['50SMA'] = hist['prev_Close'].rolling(50).mean()
    hist['200SMA'] = hist['Close'].rolling(200).mean()
    hist['50SMA'] = hist['Close'].rolling(50).mean()
    hist['50SMA_vol'] = hist['prev_Volume'].rolling(50).mean()
    

    # print(data.get_quote_yahoo(stock))
    
    display(hist.sort_values(['Date'], ascending=False))
    

[*********************100%***********************]  1 of 1 completed
{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics, and compute and networking solutions in the United States, Taiwan, China, and internationally. The company's Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building 3D designs and virtual worlds. Its Compute & Networking segment provides Data Center platforms and systems for AI, HPC, and

,Open,High,Low,Close,Adj Close,Volume,stock_float,short_interest,prev_Close,prev_Volume,200SMA,50SMA,50SMA_vol
Date,,,,,,,,,,,,,
2023-06-22,422.529999,434.260010,422.339996,430.250000,430.250000,41692600,2371027100,None,430.450012,55160300.0,216.2112,329.080401,48569746.0
2023-06-21,435.010010,436.149994,420.799988,430.450012,430.450012,55160300,2371027100,None,438.079987,45115300.0,214.7423,325.909201,48257098.0
2023-06-20,429.980011,439.899994,426.739990,438.079987,438.079987,45115300,2371027100,None,426.920013,65475500.0,213.2869,322.816001,48150100.0
2023-06-16,434.500000,437.209991,426.609985,426.920013,426.920013,65475500,2371027100,None,426.529999,56862200.0,211.8512,319.461801,47870620.0
2023-06-15,426.019989,432.890015,421.470001,426.529999,426.529999,56862200,2371027100,None,429.970001,74046500.0,210.4900,316.299601,47470560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-29,198.824997,200.987503,196.570007,200.267502,199.965988,36763200,2371027100,None,199.850006,49543600.0,NaN,NaN,NaN
2021-06-28,193.612503,200.787506,193.190002,199.850006,199.549133,49543600,2371027100,None,190.309998,27836400.0,NaN,NaN,NaN
2021-06-25,192.804993,193.455002,188.945007,190.309998,190.023468,27836400,2371027100,None,192.054993,32092400.0,NaN,NaN,NaN


In [37]:
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

api_key = "TYTNO3561Z0C1XA2"  
# Use the TimeSeries class from the alpha_vantage library to retrieve real-time data:
# Create an instance of the TimeSeries class
ts = TimeSeries(key=api_key, output_format='pandas')

# Get the real-time stock quote
symbol = "AAPL"
data, metadata = ts.get_quote_endpoint(symbol)

# Print the real-time data
print(data)

             01. symbol  02. open  03. high   04. low 05. price 06. volume  \
Global Quote       AAPL  184.4100  186.1000  184.4100  185.0100   48964426   

             07. latest trading day 08. previous close 09. change  \
Global Quote             2023-06-20           184.9200     0.0900   

             10. change percent  
Global Quote            0.0487%  


In [64]:

ts = TimeSeries(key=api_key, output_format='json')
ti = TechIndicators(key=api_key, output_format='pandas')

# Get the real-time stock quote
symbol = "NVDA"
data, metadata = ti.get_sma(symbol, interval='daily', time_period=200, series_type='close')

# data, metadata = ti.get_technical_indicator(symbol, interval='daily')

print(data)
moving_average = data['2023-06-16']

# Print the real-time data
# print("200-day Moving Average:", moving_average)

                 SMA
date                
2000-08-15    1.5550
2000-08-16    1.5667
2000-08-17    1.5785
2000-08-18    1.5906
2000-08-21    1.6028
...              ...
2023-06-13  207.7590
2023-06-14  209.0965
2023-06-15  210.4397
2023-06-16  211.8016
2023-06-20  213.2379

[5747 rows x 1 columns]


KeyError: '2023-06-16'

In [33]:
end_date = datetime.now()
end_date

datetime.datetime(2023, 6, 20, 18, 5, 40, 77602)

In [19]:
url = 'https://www.alphavantage.co/query?function=MARKET_STATUS&apikey=TYTNO3561Z0C1XA2'
response = requests.get(url)
data = json.loads(response.text)
markets = pd.json_normalize(data['markets'])

display(markets)

In [21]:
listings[['exchange','assetType']] \
    .value_counts() \
    .reset_index(name='count') \
#    .to_csv('listings_ct.csv', index=False)

In [54]:
ts = TimeSeries(key=api_key, output_format='pandas')

# Get all available technical indicators
symbol = "NVDA"  # Replace with the desired stock symbol
data, metadata = ts.get_batch_stock_quotes(symbols=symbol)

AttributeError: 'TimeSeries' object has no attribute 'get_batch_stock_quotes'

In [6]:

import requests
import pandas as pd
from bs4 import BeautifulSoup

# Get the webpage
url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001045810&type=10-K&dateb=&owner=exclude&count=40'
response = requests.get(url)

# Load the webpage content
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())

# Get the table having the filings data
table = soup.find('table', {'class': 'tableFile2'}) 

# Get all the rows of table
rows = table.find_all('tr') # find error here         

# Get the columns from the table
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
columns


# Initialize the dataframe with the columns
df = pd.DataFrame(columns=columns)


# Get all the table rows
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    # Get all the table data
    data = [td.text.replace('\n', '').strip() for td in tds]
    
    # Append the data to the dataframe
    df = df.append(pd.Series(data=data, index=columns), ignore_index=True)

# Display the dataframe
print(df)

# Get the link to the XBRL instance document
instance_document = 'https://www.sec.gov/Archives/' + df['Filings'].iloc[0]

# Get the XBRL instance document
xbrl_response = requests.get(instance_document)

# Load the XBRL instance document
xbrl_response_content = BeautifulSoup(xbrl_response.content, 'lxml')

# Find and print stockholder's equity
stockholder_equity = xbrl_response_content.find('us-gaap:stockholdersequity')

print(stockholder_equity)

# Find and print the common stock value
common_stock = xbrl_response_content.find('us-gaap:commonstockvalue')

print(common_stock)

# Find and print the retained earnings value
retained_earnings = xbrl_response_content.find('us-gaap:retainedearningsdeficit')

print(retained_earnings)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   SEC.gov | Request Rate Threshold Exceeded
  </title>
  <style>
   html {height: 100%}
body {height: 100%; margin:0; padding:0;}
#header {background-color:#003968; color:#fff; padding:15px 20px 10px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px; border-bottom:solid 5px #000;}
#footer {background-color:#003968; color:#fff; padding:15px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px;}
#content {max-width:650px;margin:60px auto; padding:0 20px 100px 20px; background-image:url(seal_bw.png);background-repeat:no-repeat;background-position:50% 100%;}
h1 {font-family:Georgia, Times, serif; font-size:20px;}
h2 {text-align:center; font-family:Georgia, Times, serif; font-size:20px; width:100%; border-bottom:solid 

AttributeError: 'NoneType' object has no attribute 'find_all'

In [27]:
data = {
	'address1': '2788 San Tomas Expressway',
	'city': 'Santa Clara',
	'state': 'CA',
	'zip': '95051',
	'country': 'United States',
	'phone': '408 486 2000',
	'website': 'https://www.nvidia.com',
	'industry': 'Semiconductors',
	'industryDisp': 'Semiconductors',
	'sector': 'Technology',
	'longBusinessSummary': "NVIDIA Corporation provides graphics, and compute and networking solutions in the United States, Taiwan, China, and internationally. The company's Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building 3D designs and virtual worlds. Its Compute & Networking segment provides Data Center platforms and systems for AI, HPC, and accelerated computing; Mellanox networking and interconnect solutions; automotive AI Cockpit, autonomous driving development agreements, and autonomous vehicle solutions; cryptocurrency mining processors; Jetson for robotics and other embedded platforms; and NVIDIA AI Enterprise and other software. The company's products are used in gaming, professional visualization, datacenter, and automotive markets. NVIDIA Corporation sells its products to original equipment manufacturers, original device manufacturers, system builders, add-in board manufacturers, retailers/distributors, independent software vendors, Internet and cloud service providers, automotive manufacturers and tier-1 automotive suppliers, mapping companies, start-ups, and other ecosystem participants. It has a strategic collaboration with Kroger Co. NVIDIA Corporation was incorporated in 1993 and is headquartered in Santa Clara, California.",
	'fullTimeEmployees': 26196,
	'companyOfficers': [{
		'maxAge': 1,
		'name': 'Mr. Jen-Hsun  Huang',
		'age': 59,
		'title': 'Co-Founder, CEO, Pres & Director',
		'yearBorn': 1963,
		'fiscalYear': 2023,
		'totalPay': 1690542,
		'exercisedValue': 442805504,
		'unexercisedValue': 94833752
	}, {
		'maxAge': 1,
		'name': 'Ms. Colette M. Kress',
		'age': 55,
		'title': 'Exec. VP & CFO',
		'yearBorn': 1967,
		'fiscalYear': 2023,
		'totalPay': 912551,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Ms. Debora  Shoquist',
		'age': 67,
		'title': 'Exec. VP of Operations',
		'yearBorn': 1955,
		'fiscalYear': 2023,
		'totalPay': 870785,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Mr. Timothy S. Teter',
		'age': 55,
		'title': 'Exec. VP, Gen. Counsel & Sec.',
		'yearBorn': 1967,
		'fiscalYear': 2023,
		'totalPay': 862709,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Mr. Ajay K. Puri',
		'age': 67,
		'title': 'Exec. VP of Worldwide Field Operations',
		'yearBorn': 1955,
		'fiscalYear': 2023,
		'totalPay': 993707,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Mr. Chris A. Malachowsky',
		'title': 'Co-Founder',
		'fiscalYear': 2002,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Mr. Donald F. Robertson Jr.',
		'age': 53,
		'title': 'VP & Chief Accounting Officer',
		'yearBorn': 1969,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Prof. William J. Dally',
		'age': 61,
		'title': 'Chief Scientist & Sr. VP of Research',
		'yearBorn': 1961,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Ms. Simona  Jankowski C.F.A., J.D.',
		'title': 'VP of Investor Relations',
		'exercisedValue': 0,
		'unexercisedValue': 0
	}, {
		'maxAge': 1,
		'name': 'Mr. Robert  Sherbin',
		'age': 63,
		'title': 'VP of Corp. Communications',
		'yearBorn': 1959,
		'exercisedValue': 0,
		'unexercisedValue': 0
	}],
	'auditRisk': 5,
	'boardRisk': 9,
	'compensationRisk': 1,
	'shareHolderRightsRisk': 8,
	'overallRisk': 7,
	'governanceEpochDate': 1685577600,
	'compensationAsOfEpochDate': 1703980800,
	'maxAge': 86400,
	'priceHint': 2,
	'previousClose': 430.45,
	'open': 422.53,
	'dayLow': 422.34,
	'dayHigh': 434.255,
	'regularMarketPreviousClose': 430.45,
	'regularMarketOpen': 422.53,
	'regularMarketDayLow': 422.34,
	'regularMarketDayHigh': 434.255,
	'dividendRate': 0.16,
	'dividendYield': 0.0004,
	'exDividendDate': 1686096000,
	'payoutRatio': 0.0833,
	'fiveYearAvgDividendYield': 0.18,
	'beta': 1.751408,
	'trailingPE': 222.92746,
	'forwardPE': 41.4499,
	'volume': 41655746,
	'regularMarketVolume': 41655746,
	'averageVolume': 47002012,
	'averageVolume10days': 52304250,
	'averageDailyVolume10Day': 52304250,
	'bid': 0.0,
	'ask': 0.0,
	'bidSize': 1100,
	'askSize': 1300,
	'marketCap': 1062717554688,
	'fiftyTwoWeekLow': 108.13,
	'fiftyTwoWeekHigh': 439.9,
	'priceToSalesTrailing12Months': 41.06645,
	'fiftyDayAverage': 329.0804,
	'twoHundredDayAverage': 216.2112,
	'trailingAnnualDividendRate': 0.16,
	'trailingAnnualDividendYield': 0.00037170402,
	'currency': 'USD',
	'enterpriseValue': 1059971530752,
	'profitMargins': 0.18521,
	'floatShares': 2371027100,
	'sharesOutstanding': 2470000128,
	'sharesShort': 26884245,
	'sharesShortPriorMonth': 32599217,
	'sharesShortPreviousMonthDate': 1682640000,
	'dateShortInterest': 1685491200,
	'sharesPercentSharesOut': 0.0109,
	'heldPercentInsiders': 0.041810002,
	'heldPercentInstitutions': 0.68084,
	'shortRatio': 0.51,
	'shortPercentOfFloat': 0.0113,
	'impliedSharesOutstanding': 2470000128,
	'bookValue': 9.915,
	'priceToBook': 43.39385,
	'lastFiscalYearEnd': 1674950400,
	'nextFiscalYearEnd': 1706486400,
	'mostRecentQuarter': 1682812800,
	'earningsQuarterlyGrowth': 0.263,
	'netIncomeToCommon': 4792999936,
	'trailingEps': 1.93,
	'forwardEps': 10.38,
	'pegRatio': 2.61,
	'lastSplitFactor': '4:1',
	'lastSplitDate': 1626739200,
	'enterpriseToRevenue': 40.96,
	'enterpriseToEbitda': 174.051,
	'52WeekChange': 1.5122621,
	'SandP52WeekChange': 0.12018943,
	'lastDividendValue': 0.04,
	'lastDividendDate': 1686096000,
	'exchange': 'NMS',
	'quoteType': 'EQUITY',
	'symbol': 'NVDA',
	'underlyingSymbol': 'NVDA',
	'shortName': 'NVIDIA Corporation',
	'longName': 'NVIDIA Corporation',
	'firstTradeDateEpochUtc': 917015400,
	'timeZoneFullName': 'America/New_York',
	'timeZoneShortName': 'EDT',
	'uuid': '7f5f6a07-b148-30f4-98a2-2caa3df2aed0',
	'messageBoardId': 'finmb_32307',
	'gmtOffSetMilliseconds': -14400000,
	'currentPrice': 430.25,
	'targetHighPrice': 710.0,
	'targetLowPrice': 244.8,
	'targetMeanPrice': 467.95,
	'targetMedianPrice': 462.0,
	'recommendationMean': 1.9,
	'recommendationKey': 'buy',
	'numberOfAnalystOpinions': 40,
	'totalCash': 15320000512,
	'totalCashPerShare': 6.202,
	'ebitda': 6089999872,
	'totalDebt': 12080000000,
	'quickRatio': 2.672,
	'currentRatio': 3.427,
	'totalRevenue': 25877999616,
	'debtToEquity': 49.266,
	'revenuePerShare': 10.443,
	'returnOnAssets': 0.06267,
	'returnOnEquity': 0.18855,
	'grossProfits': 15356000000,
	'freeCashflow': 5470499840,
	'operatingCashflow': 6821000192,
	'earningsGrowth': 0.281,
	'revenueGrowth': -0.132,
	'grossMargins': 0.56314,
	'ebitdaMargins': 0.23534,
	'operatingMargins': 0.17374,
	'financialCurrency': 'USD',
	'trailingPegRatio': 2.4458
}

In [32]:
del data['companyOfficers']
df = pd.DataFrame.from_dict(data)

KeyError: 'companyOfficers'

In [33]:
df = pd.DataFrame.from_dict(data)

ValueError: If using all scalar values, you must pass an index

In [35]:
df['companyOfficers']

0    {'maxAge': 1, 'name': 'Mr. Jen-Hsun  Huang', '...
1    {'maxAge': 1, 'name': 'Ms. Colette M. Kress', ...
2    {'maxAge': 1, 'name': 'Ms. Debora  Shoquist', ...
3    {'maxAge': 1, 'name': 'Mr. Timothy S. Teter', ...
4    {'maxAge': 1, 'name': 'Mr. Ajay K. Puri', 'age...
5    {'maxAge': 1, 'name': 'Mr. Chris A. Malachowsk...
6    {'maxAge': 1, 'name': 'Mr. Donald F. Robertson...
7    {'maxAge': 1, 'name': 'Prof. William J. Dally'...
8    {'maxAge': 1, 'name': 'Ms. Simona  Jankowski C...
9    {'maxAge': 1, 'name': 'Mr. Robert  Sherbin', '...
Name: companyOfficers, dtype: object